### 1. Importing the Libraries

In [3]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### 2. Downloading The Images

In [4]:
data = []
labels = []

for tumor_type in ['yes', 'no']:
    for r, d, f in os.walk(f"bE:\Ruhama's Project\Tumor Detection with CNN\brain_tumor_dataset{tumor_type}"):
        for file in f:
            
            _, ext = os.path.splitext(file)
            
            if ext.lower() in ['.jpg', '.jpeg', '.png']:
                # File path
                path = os.path.join(r, file)
                
                # read the image with OpenCV
                img = cv2.imread(path)
                
                # resize the image
                img = cv2.resize(img, (128, 128))
                
                # convert to numpy array
                img = np.array(img)
                
                # if the size of the image is correct
                if img.shape == (128, 128, 3):
                    data.append(img)
                    
                    label = 0 if tumor_type == 'yes' else 1
                    labels.append(label)


### 3. Data Visualization

In [6]:
num_samples_to_display = 5
random_indices = np.random.choice(len(data), num_samples_to_display, replace=False)

plt.figure(figsize=(10, 5))
for i, idx in enumerate(random_indices):
    plt.subplot(1, num_samples_to_display, i + 1)
    plt.imshow(data[idx], interpolation='nearest')
    plt.title('Tumor' if labels[idx] == 0 else 'No Tumor')
    plt.axis('off')
plt.show()

ValueError: a must be greater than 0 unless no samples are taken

### 4. Split The Data 

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size= 0.3, random_state= 42)

print(x_train.shape)
print(x_test.shape)

### 5. Data Distribution

In [ ]:
#Train
plt.figure(figsize=(4, 3))
plt.hist(y_train, bins=np.arange(3)-0.5, alpha=0.5, color='blue', edgecolor='black', linewidth=1.2)
plt.xticks([0, 1])  
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Train Data Class Distribution')
plt.grid(True)
plt.show()

#Test
plt.figure(figsize=(4, 3))
plt.hist(y_test, bins=np.arange(3)-0.5, alpha=0.5, color='red', edgecolor='black', linewidth=1.2)
plt.xticks([0, 1])
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Test Data Class Distribution')
plt.grid(True)
plt.show()

### 6. Data Preprocessing

In [ ]:
def preProcess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    
    return img

In [ ]:
x_train = np.array(list(map(preProcess, x_train)))
x_test = np.array(list(map(preProcess, x_test)))

In [ ]:
x_train = x_train.reshape(-1, 128, 128,1)
x_test = x_test.reshape(-1, 128, 128,1)

### 6. Data Augmentation 

In [ ]:
dataGen = ImageDataGenerator(width_shift_range= 0.1,
                             height_shift_range= 0.1,
                             zoom_range= 0.1,
                             rotation_range= 10)

In [ ]:
dataGen.fit(x_train)

In [ ]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

### 7. CNN Model

In [ ]:
model = Sequential()
model.add(Conv2D(32,input_shape = (128,128,1), kernel_size = (5,5), activation="relu", padding = "same"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units= 2, activation="softmax"))

model.compile(loss = "categorical_crossentropy", optimizer = ("Adam"), metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, epochs = 30, batch_size = 40, verbose = 1, validation_data = (x_test, y_test))

### 8. Results

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label = "Train Loss")
plt.plot(history.history["val_loss"], label = "Val Loss")
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history["accuracy"], label = "Train accuracy")
plt.plot(history.history["val_accuracy"], label = "Val accuracy")
plt.legend()
plt.show()

In [ ]:
score = model.evaluate(x_test,y_test, verbose = 1)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

In [ ]:
y_pred = model.predict(x_test)

y_pred_class = np.argmax(y_pred, axis = 1)

Y_true = np.argmax(y_test, axis= 1)

cm = confusion_matrix(Y_true, y_pred_class)

f, ax = plt.subplots(figsize = (4,4))
sns.heatmap(cm, annot= True, linewidths= 0.01, cmap = "Greens", linecolor="gray", fmt= ".1f", ax = ax)
plt.xlabel("predicted")
plt.ylabel("true")
plt.title("cm")
plt.show()